# Import stuff

In [1]:
import os
import sys
sys.path.append(os.path.abspath('') + '/../../..')

In [2]:
# from dataset_evaluation_utils import *

from datetime import datetime
import pandas as pd 


___
# Read Data

In [3]:
# importa dataset 'movieles'
data = pd.read_csv('../../output/movielens_dump/sampled_movielens.csv')
user_col = 'UserID'
item_col = 'ItemID'

### Convert timestamp

In [4]:
%%time 
# 2.42s
data['date'] = data['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %X'))
# data.sort_values(by='timestamp', inplace=True)

CPU times: user 367 ms, sys: 27.2 ms, total: 394 ms
Wall time: 393 ms


# EXPERIMENT 7

CODE TO GET LAST N INTERACTIONS FROM EACH USER AS HOLDOUT  
* IF USER DID NOT INTERACT WITH AT LEAST N+1 ITEMS, THEN IT IS NOT USED FOR HOLDOUT  
* LAST 10 INTERACTIONS FROM EACH USER AS HOLDOUT
* RECOMENDING SEEN ITEMS IS ALLOWED
* DON'T REMOVE INTERACTIONS IN HOLDOUT FROM BUCKETS
* DON'T REMOVE INTERACTIONS **SENT** TO HOLDOUT FROM BUCKET

In [5]:
# CODE TO GET LAST N INTERACTIONS FROM EACH USER AS HOLDOUT
# IF USER DID NOT INTERACT WITH AT LEAST N+1 ITEMS, THEN IT IS NOT USED FOR HOLDOUT

N = 10
cold_start_buckets = 0
#     print('0',data.shape[0]) # debug
print('Creating buckets. . .')
buckets = []
# assert interval_type in ['W', 'M', 'QS', 'F'], "interval must be one of W, M, QS, or F"
# create buckets based on months
months = data['date'].unique()
months.sort()
for interval in months:
    idx = (data['date'] == interval)
    buckets.append( data[idx] )

Creating buckets. . .


In [6]:
print('Creating holdouts. . .')
# create holdouts with last user interaction
holdouts = []

for i, b in enumerate( buckets ):
    if i >= cold_start_buckets:
        condition = (b[user_col].value_counts() > N)
        frequent_users = b[user_col].value_counts()[ condition ].index
        holdout_idx = []
        for u in frequent_users:
            tail_idx = list( b[b[user_col] == u].tail(N).index )
            holdout_idx += tail_idx
        holdout = b.loc[holdout_idx].reset_index(drop=True)
        holdouts.append(holdout)
        # buckets[i] = b.drop(index=holdout_idx).reset_index(drop=True)
        buckets[i] = b.reset_index(drop=True)

Creating holdouts. . .


Store SAMPLE buckets and holdouts

In [7]:
buckets = [b.iloc[:1000] for b in buckets]

In [8]:
holdouts = [h.iloc[:500] for h in holdouts]

In [9]:
print([b.shape for b in buckets])
print([h.shape for h in holdouts])

[(1000, 7), (1000, 7), (1000, 7)]
[(500, 7), (500, 7), (500, 7)]


In [10]:
for bi in range(len(buckets)):
    print(pd.concat(buckets[:bi+1]).shape)
    pd.concat(buckets[:bi+1]).to_csv(f'movielens_b{bi}.csv', sep='\t', columns=[user_col, item_col], header=False, index=False)
    holdouts[bi].to_csv(f'movielens_h{bi}.csv', sep='\t', columns=[user_col, item_col], header=False, index=False)

(1000, 7)
(2000, 7)
(3000, 7)
